In [6]:
# Importing Required Libraries

import os
from bs4 import BeautifulSoup
import requests
import pandas as pd
from pandas import Series, DataFrame
from IPython.html.widgets import FloatProgress
from time import sleep
from IPython.display import display
from requests.exceptions import ConnectionError
import numpy as np
import time
pd.set_option('display.expand_frame_repr', False)
import urllib
from selenium import webdriver  
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary

In [8]:
url = 'http://www.jcpenney.com/g/refrigerators/N-1b2ke21Dgn3fgy?productGridView=large'
pgurl_fore = 'http://www.jcpenney.com/g/refrigerators/N-1b2ke21Dgn3fgy?page='
pgurl_back = '&merchSize=2'
numpgs = 10
! mkdir refrigerators
entity = "refrigerators"


In [9]:
# Defning variables 
title = []
image_url = []
image_name = []

In [10]:
# Function to for scraping. Function uses Selenium to scrape. It is required to skip the Javascript call
def makesoup(url,ini):
    
#     Added lines for incorporating selenium, provide the path where you have geckodriver.exe file
    PROJECT_PATH = os.path.dirname(os.path.dirname("_PATH_"))

    browser = webdriver.Firefox(PROJECT_PATH) 
    browser.get(url)  
    html_source = browser.page_source  
    browser.quit()
#     End of selenium part
    
    soup = BeautifulSoup(html_source, "html.parser")
    i = ini-1
    for each in soup.findAll('li',{'class':'productDisplay'}):
        i = i+1
#         For fetching the title
        if each.find('div',{'class':'productDisplay_title'}) and each.find('img',{'class':'productDisplay_image--default'}):
            
            title.append(each.find('div',{'class':'productDisplay_title'}).find(text=True).strip())
            imgurlt = each.find('img',{'class':'productDisplay_image--default'}).get('src')
            image_url.append(imgurlt)
            urllib.urlretrieve(imgurlt, ".\\" + entity + "\\img_"+str(i)+".jpg")
            image_name.append("img_"+str(i)+".jpg")
            
        else:
            title.append("NA")
            image_url.append("NA")
            image_name.append("NA")
        
    return i

In [11]:
max = numpgs

try:
    fut = makesoup(url,ini=0) 
except ConnectionError:
    time.sleep(1200)
    fut = makesoup(url,ini=fut) 
fut = fut+1
    
# For locaing bar
f = FloatProgress(min=1, max=max-1)
display(f)

for j in range(2,max):
    f.value = j
    try:
        fut = makesoup(pgurl_fore + str(j) + pgurl_back,ini=fut) 
    except ConnectionError:
        time.sleep(1200)
        fut = makesoup(pgurl_fore + str(j) + pgurl_back,ini=fut) 
    time.sleep(10)
    fut = fut + 1


In [12]:
# Converting to dF and exporting
title = Series(title)
image_url = Series(image_url)
image_name = Series(image_name)
tdf = pd.concat([title ,image_url ,image_name ],axis=1)
tdf.columns = ["title ","image_url ","image_name "]
tdf.to_csv(".\\"+ entity + "\\jcp_" + entity + ".txt", sep='\t', encoding='utf-8')